In [1]:
# This code is used for JHU CS 482/682: Deep Learning 2019 Spring Project
# Copyright: Zhaoshuo Li, Ding Hao, Mingyi Zheng
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as functional
from torch.utils.data import DataLoader, Dataset
# from torchvision import transforms
import torchvision.transforms.functional as TF

import numpy as np
import random
from tensorboardX import SummaryWriter

import transforms
from dataset import *
from visualization import *
from label_conversion import *
from dice_loss import *
from model_trainning import *
from model_pretrainning import *
from model_from_ternaus import *

from unet import *

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="2"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

cuda


# Seed pytorch and numpy and random

In [3]:
# IMPORTANT!
# must seed the same value each time when training a new network
seed = 256
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
pretrain_seed = 128

## Hyperparameters

In [4]:
train_batch_size = 8
validation_batch_size= 8
learning_rate = 0.001
num_epochs = 150
num_class = 12

In [5]:
weights = torch.ones((num_class,1))
weights = weights.to(device)
dice_loss = DICELoss(weights) 

## Visualization

In [6]:
# Initialize the visualization environment
writer = SummaryWriter()

# Baseline, without augmentation

### Unet

In [7]:
# IMPORTANT!
# must seed the same value each time when training a new network
seed = 256
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

In [8]:
# initialize model
model = unet(useBN=True)
model.to(device)
print("model loaded")

model loaded


### Optimizer and Scheduler and loss

In [9]:
# intialize optimizer and lr decay
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

In [10]:
# define dataset
train_dataset=MICCAIDataset(data_type = "train", transform=None)
validation_dataset=MICCAIDataset(data_type = "validation", transform=None)
label_converter = LabelConverter()

# # show one example
# img,label = train_dataset.__getitem__(0)
# imshow(img.permute(1,2,0),denormalize=True)
# colorlabel = train_dataset.label_converter.label2color(label.permute(1,2,0))
# imshow(colorlabel)

# # show one example
# img,label = validation_dataset.__getitem__(0)
# imshow(img.permute(1,2,0),denormalize=True)
# colorlabel = train_dataset.label_converter.label2color(label.permute(1,2,0))
# imshow(colorlabel)

# intialize the dataloader
train_generator = DataLoader(train_dataset,shuffle=True,batch_size=train_batch_size,num_workers=8)
validation_generator = DataLoader(validation_dataset,shuffle=True,batch_size=validation_batch_size,num_workers=8)

### Start training

In [11]:
best_model_wts, best_accuracy = run_training(model,device,num_class,scheduler,optimizer,dice_loss,num_epochs,train_generator,train_dataset,validation_generator,validation_dataset,writer)
print(best_accuracy)

Training Started!

EPOCH 1 of 150

Training Loss: 0.8568
0 Class, True Pos 29171220.0, False Pos 21813304.0, False Neg 23953550.0, Num Pixel 53124768.0, Dice score 0.56
1 Class, True Pos 6685966.0, False Pos 5457427.0, False Neg 4237535.0, Num Pixel 10923501.0, Dice score 0.58
2 Class, True Pos 3256451.0, False Pos 5489278.0, False Neg 2101208.0, Num Pixel 5357659.0, Dice score 0.46
3 Class, True Pos 296683.0, False Pos 1765565.0, False Neg 2180660.0, Num Pixel 2477343.0, Dice score 0.13
4 Class, True Pos 846285.0, False Pos 1868685.0, False Neg 19203332.0, Num Pixel 20049616.0, Dice score 0.07
5 Class, True Pos 7814052.0, False Pos 18079506.0, False Neg 5882046.0, Num Pixel 13696098.0, Dice score 0.39
6 Class, True Pos 15521.0, False Pos 3497506.0, False Neg 403097.0, Num Pixel 418618.0, Dice score 0.01
7 Class, True Pos 265.0, False Pos 5073.0, False Neg 612899.0, Num Pixel 613164.0, Dice score 0.00
8 Class, True Pos 0.0, False Pos 1615176.0, False Neg 3960.0, Num Pixel 3960.0, Dice 

Vaildation Loss: 0.5557
0 Class, True Pos 10398099.0, False Pos 4040756.0, False Neg 2707594.0, Num Pixel 13105693.0, Dice score 0.76
1 Class, True Pos 1911802.0, False Pos 534941.0, False Neg 627397.0, Num Pixel 2539199.0, Dice score 0.77
2 Class, True Pos 775114.0, False Pos 350286.0, False Neg 560316.0, Num Pixel 1335430.0, Dice score 0.63
3 Class, True Pos 307212.0, False Pos 181996.0, False Neg 256558.0, Num Pixel 563770.0, Dice score 0.58
4 Class, True Pos 2405896.0, False Pos 2245755.0, False Neg 2531623.0, Num Pixel 4937519.0, Dice score 0.50
5 Class, True Pos 2246766.0, False Pos 1547777.0, False Neg 1247033.0, Num Pixel 3493799.0, Dice score 0.62
6 Class, True Pos 41377.0, False Pos 31774.0, False Neg 52809.0, Num Pixel 94186.0, Dice score 0.49
7 Class, True Pos 87859.0, False Pos 28490.0, False Neg 51663.0, Num Pixel 139522.0, Dice score 0.69
8 Class, True Pos 0.0, False Pos 0.0, False Neg 499.0, Num Pixel 499.0, Dice score 0.00
9 Class, True Pos 0.0, False Pos 0.0, False Ne

best performance: dice 0.65
Dice Score: 0.5232
----------

EPOCH 8 of 150

Training Loss: 0.4898
0 Class, True Pos 43513036.0, False Pos 13238107.0, False Neg 9611741.0, Num Pixel 53124776.0, Dice score 0.79
1 Class, True Pos 8418427.0, False Pos 1397356.0, False Neg 2505074.0, Num Pixel 10923501.0, Dice score 0.81
2 Class, True Pos 3329559.0, False Pos 1298021.0, False Neg 2028100.0, Num Pixel 5357659.0, Dice score 0.67
3 Class, True Pos 1630562.0, False Pos 682874.0, False Neg 846781.0, Num Pixel 2477343.0, Dice score 0.68
4 Class, True Pos 12208892.0, False Pos 6926773.0, False Neg 7840723.0, Num Pixel 20049616.0, Dice score 0.62
5 Class, True Pos 9112546.0, False Pos 3940556.0, False Neg 4583552.0, Num Pixel 13696098.0, Dice score 0.68
6 Class, True Pos 239387.0, False Pos 139479.0, False Neg 179231.0, Num Pixel 418618.0, Dice score 0.60
7 Class, True Pos 415755.0, False Pos 161637.0, False Neg 197409.0, Num Pixel 613164.0, Dice score 0.70
8 Class, True Pos 0.0, False Pos 0.0, Fals

Vaildation Loss: 0.4490
0 Class, True Pos 11722474.0, False Pos 2995817.0, False Neg 1383219.0, Num Pixel 13105693.0, Dice score 0.84
1 Class, True Pos 1864012.0, False Pos 175121.0, False Neg 675187.0, Num Pixel 2539199.0, Dice score 0.81
2 Class, True Pos 833341.0, False Pos 259640.0, False Neg 502089.0, Num Pixel 1335430.0, Dice score 0.69
3 Class, True Pos 444522.0, False Pos 259851.0, False Neg 119248.0, Num Pixel 563770.0, Dice score 0.70
4 Class, True Pos 3227858.0, False Pos 825654.0, False Neg 1709661.0, Num Pixel 4937519.0, Dice score 0.72
5 Class, True Pos 2792930.0, False Pos 1010248.0, False Neg 700869.0, Num Pixel 3493799.0, Dice score 0.77
6 Class, True Pos 45883.0, False Pos 21915.0, False Neg 48303.0, Num Pixel 94186.0, Dice score 0.57
7 Class, True Pos 116847.0, False Pos 34930.0, False Neg 22675.0, Num Pixel 139522.0, Dice score 0.80
8 Class, True Pos 0.0, False Pos 0.0, False Neg 499.0, Num Pixel 499.0, Dice score 0.00
9 Class, True Pos 64803.0, False Pos 85078.0, F

best performance: dice 0.82
Dice Score: 0.6997
----------

EPOCH 15 of 150

Training Loss: 0.3780
0 Class, True Pos 47957704.0, False Pos 10062353.0, False Neg 5167072.0, Num Pixel 53124776.0, Dice score 0.86
1 Class, True Pos 9314801.0, False Pos 1132718.0, False Neg 1608700.0, Num Pixel 10923501.0, Dice score 0.87
2 Class, True Pos 3768810.0, False Pos 1105915.0, False Neg 1588849.0, Num Pixel 5357659.0, Dice score 0.74
3 Class, True Pos 1816123.0, False Pos 492607.0, False Neg 661220.0, Num Pixel 2477343.0, Dice score 0.76
4 Class, True Pos 15041330.0, False Pos 3575717.0, False Neg 5008285.0, Num Pixel 20049616.0, Dice score 0.78
5 Class, True Pos 10621367.0, False Pos 2106581.0, False Neg 3074731.0, Num Pixel 13696098.0, Dice score 0.80
6 Class, True Pos 266292.0, False Pos 103939.0, False Neg 152326.0, Num Pixel 418618.0, Dice score 0.68
7 Class, True Pos 480376.0, False Pos 70225.0, False Neg 132788.0, Num Pixel 613164.0, Dice score 0.83
8 Class, True Pos 0.0, False Pos 0.0, Fal

...

best performance: dice 0.94
----------

EPOCH 134 of 150

Training Loss: 0.2575
0 Class, True Pos 51026176.0, False Pos 3239925.0, False Neg 2098582.0, Num Pixel 53124760.0, Dice score 0.95
1 Class, True Pos 10331882.0, False Pos 531379.0, False Neg 591619.0, Num Pixel 10923501.0, Dice score 0.95
2 Class, True Pos 4770814.0, False Pos 607103.0, False Neg 586845.0, Num Pixel 5357659.0, Dice score 0.89
3 Class, True Pos 2175031.0, False Pos 206038.0, False Neg 302312.0, Num Pixel 2477343.0, Dice score 0.90
4 Class, True Pos 18726328.0, False Pos 960811.0, False Neg 1323284.0, Num Pixel 20049612.0, Dice score 0.94
5 Class, True Pos 12732387.0, False Pos 634522.0, False Neg 963711.0, Num Pixel 13696098.0, Dice score 0.94
6 Class, True Pos 326703.0, False Pos 78704.0, False Neg 91915.0, Num Pixel 418618.0, Dice score 0.79
7 Class, True Pos 556391.0, False Pos 36686.0, False Neg 56773.0, Num Pixel 613164.0, Dice score 0.92
8 Class, True Pos 0.0, False Pos 0.0, False Neg 3960.0, Num Pixel 396

Vaildation Loss: 0.2981
0 Class, True Pos 12455230.0, False Pos 896837.0, False Neg 650463.0, Num Pixel 13105693.0, Dice score 0.94
1 Class, True Pos 2386996.0, False Pos 190956.0, False Neg 152203.0, Num Pixel 2539199.0, Dice score 0.93
2 Class, True Pos 1144247.0, False Pos 187482.0, False Neg 191183.0, Num Pixel 1335430.0, Dice score 0.86
3 Class, True Pos 470217.0, False Pos 83107.0, False Neg 93553.0, Num Pixel 563770.0, Dice score 0.84
4 Class, True Pos 4575399.0, False Pos 295294.0, False Neg 362120.0, Num Pixel 4937519.0, Dice score 0.93
5 Class, True Pos 3229444.0, False Pos 186250.0, False Neg 264355.0, Num Pixel 3493799.0, Dice score 0.93
6 Class, True Pos 65083.0, False Pos 22676.0, False Neg 29103.0, Num Pixel 94186.0, Dice score 0.72
7 Class, True Pos 125621.0, False Pos 13362.0, False Neg 13901.0, Num Pixel 139522.0, Dice score 0.90
8 Class, True Pos 0.0, False Pos 0.0, False Neg 499.0, Num Pixel 499.0, Dice score 0.00
9 Class, True Pos 86768.0, False Pos 8781.0, False N

best performance: dice 0.94
----------

EPOCH 141 of 150

Training Loss: 0.2622
0 Class, True Pos 51005412.0, False Pos 3183344.0, False Neg 2119369.0, Num Pixel 53124780.0, Dice score 0.95
1 Class, True Pos 10325195.0, False Pos 525130.0, False Neg 598306.0, Num Pixel 10923501.0, Dice score 0.95
2 Class, True Pos 4776757.0, False Pos 596668.0, False Neg 580902.0, Num Pixel 5357659.0, Dice score 0.89
3 Class, True Pos 2181780.0, False Pos 209218.0, False Neg 295563.0, Num Pixel 2477343.0, Dice score 0.90
4 Class, True Pos 18770884.0, False Pos 969002.0, False Neg 1278729.0, Num Pixel 20049612.0, Dice score 0.94
5 Class, True Pos 12747238.0, False Pos 620965.0, False Neg 948860.0, Num Pixel 13696098.0, Dice score 0.94
6 Class, True Pos 329069.0, False Pos 80522.0, False Neg 89549.0, Num Pixel 418618.0, Dice score 0.79
7 Class, True Pos 555604.0, False Pos 35911.0, False Neg 57560.0, Num Pixel 613164.0, Dice score 0.92
8 Class, True Pos 0.0, False Pos 0.0, False Neg 3960.0, Num Pixel 396

Vaildation Loss: 0.3089
0 Class, True Pos 12459084.0, False Pos 896534.0, False Neg 646609.0, Num Pixel 13105693.0, Dice score 0.94
1 Class, True Pos 2383133.0, False Pos 181561.0, False Neg 156066.0, Num Pixel 2539199.0, Dice score 0.93
2 Class, True Pos 1145627.0, False Pos 189488.0, False Neg 189803.0, Num Pixel 1335430.0, Dice score 0.86
3 Class, True Pos 467427.0, False Pos 79268.0, False Neg 96343.0, Num Pixel 563770.0, Dice score 0.84
4 Class, True Pos 4575370.0, False Pos 293932.0, False Neg 362149.0, Num Pixel 4937519.0, Dice score 0.93
5 Class, True Pos 3247483.0, False Pos 201561.0, False Neg 246316.0, Num Pixel 3493799.0, Dice score 0.94
6 Class, True Pos 63089.0, False Pos 19902.0, False Neg 31097.0, Num Pixel 94186.0, Dice score 0.71
7 Class, True Pos 124108.0, False Pos 11567.0, False Neg 15414.0, Num Pixel 139522.0, Dice score 0.90
8 Class, True Pos 0.0, False Pos 0.0, False Neg 499.0, Num Pixel 499.0, Dice score 0.00
9 Class, True Pos 86176.0, False Pos 8200.0, False N

best performance: dice 0.96
----------

EPOCH 148 of 150

Training Loss: 0.2534
0 Class, True Pos 50980800.0, False Pos 3183960.0, False Neg 2143977.0, Num Pixel 53124776.0, Dice score 0.95
1 Class, True Pos 10333884.0, False Pos 535417.0, False Neg 589617.0, Num Pixel 10923501.0, Dice score 0.95
2 Class, True Pos 4773912.0, False Pos 595164.0, False Neg 583747.0, Num Pixel 5357659.0, Dice score 0.89
3 Class, True Pos 2182068.0, False Pos 211280.0, False Neg 295275.0, Num Pixel 2477343.0, Dice score 0.90
4 Class, True Pos 18771192.0, False Pos 981532.0, False Neg 1278427.0, Num Pixel 20049620.0, Dice score 0.94
5 Class, True Pos 12745631.0, False Pos 627458.0, False Neg 950467.0, Num Pixel 13696098.0, Dice score 0.94
6 Class, True Pos 327455.0, False Pos 80359.0, False Neg 91163.0, Num Pixel 418618.0, Dice score 0.79
7 Class, True Pos 554661.0, False Pos 35933.0, False Neg 58503.0, Num Pixel 613164.0, Dice score 0.92
8 Class, True Pos 0.0, False Pos 0.0, False Neg 3960.0, Num Pixel 396

In [12]:
## load best model weights
model.load_state_dict(best_model_wts)
## save model
torch.save(model.state_dict(), 'vanilla_trained_unet_2.pt')

# Augmentation

In [ ]:
# IMPORTANT!
# must seed the same value each time when training a new network
seed = 256
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
pretrain_seed = 128

In [ ]:
# initialize model
model = unet(useBN=True)
model.to(device)
print("model loaded")

In [ ]:
# intialize optimizer and lr decay
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)